# Interacting with Kinetica and the SQLAssist LLM
### Install required packages

In [ ]:
!pip install ipykernel ipywidgets gpudb langchain langchain_openai nemollm==0.3.5 colorlog "langchain-kinetica @ git+https://git@github.com/kineticadb/langchain-kinetica.git"

### Connect to Kinetica and the LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_kinetica import KineticaChatLLM, KineticaSqlOutputParser, SqlResponse

# create the Kinetica connection
kdbc = KineticaChatLLM._create_kdbc(host="https://demo72.kinetica.com/_gpudb", login="gtc", password="Kinetica123!")

# create the Kinetica LLM
kinetica_llm = KineticaChatLLM(kdbc=kdbc)

# Set the context to use
kinetica_ctx = 'raceday.raceday_ui_ctxt'

### Set up the context

In [ ]:
# load the context from the database
ctx_messages = kinetica_llm.load_messages_from_context(kinetica_ctx)

# Add the input prompt. This is where input question will be substituted.
ctx_messages.append(("human", "{input}"))

# Create the prompt template.
prompt_template = ChatPromptTemplate.from_messages(ctx_messages)
prompt_template.pretty_print()

# create the chain. 
# note: The KineticaSqlOutputParser will execute the SQL statement and is optional.
chain = prompt_template | kinetica_llm | KineticaSqlOutputParser(kdbc=kdbc)

### A simple question

In [74]:
from IPython.display import display, HTML

# Here you must ask a question relevant to the LLM context provided in the prompt template.
response: SqlResponse = chain.invoke({"input": "show me 5 videos where a crash is likely to occur"})
display(HTML(response.dataframe.to_html(index=False)))

ts,videopath
2023-10-30 19:27:46,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-15.mp4
2023-10-30 19:27:51,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-16.mp4
2023-10-30 19:28:46,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-27.mp4
2023-10-30 19:29:26,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-35.mp4
2023-10-30 19:42:25,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-3-50.mp4


# Vector Similarity Search
First, we need to generate our vector embeddings.  For this exercise, we will be using the [catch22 library](https://time-series-features.gitbook.io/catch22-features/), which is, as you'd expect, a collection of 22 different features specifically focused on time-series data.

In [ ]:
import pycatch22
import gpudb
import json
import math

def c22(ts_data: list) -> list:
    timeseries = [pycatch22.CO_f1ecac(ts_data),
                  pycatch22.CO_trev_1_num(ts_data),
                  pycatch22.CO_FirstMin_ac(ts_data),
                  pycatch22.CO_HistogramAMI_even_2_5(ts_data),
                  pycatch22.DN_Mean(ts_data),
                  pycatch22.DN_Spread_Std(ts_data),
                  pycatch22.DN_HistogramMode_5(ts_data),
                  pycatch22.DN_HistogramMode_10(ts_data),
                  pycatch22.DN_OutlierInclude_n_001_mdrmd(ts_data),
                  pycatch22.SB_BinaryStats_diff_longstretch0(ts_data),
                  pycatch22.SB_BinaryStats_mean_longstretch1(ts_data),
                  pycatch22.SB_MotifThree_quantile_hh(ts_data),
                  pycatch22.SB_TransitionMatrix_3ac_sumdiagcov(ts_data),
                  pycatch22.SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1(ts_data),
                  pycatch22.SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1(ts_data),
                  pycatch22.SP_Summaries_welch_rect_area_5_1(ts_data),
                  pycatch22.SP_Summaries_welch_rect_centroid(ts_data),
                  pycatch22.FC_LocalSimple_mean1_tauresrat(ts_data),
                  pycatch22.FC_LocalSimple_mean3_stderr(ts_data),
                  pycatch22.IN_AutoMutualInfoStats_40_gaussian_fmmi(ts_data),
                  pycatch22.MD_hrv_classic_pnn40(ts_data),
                  pycatch22.PD_PeriodicityWang_th0_01(ts_data)]

    return timeseries

### Connect to Kinetica

In [ ]:
db = gpudb.GPUdb(host='https://demo72.kinetica.com/_gpudb',
                 username='gtc',
                 password='Kinetica123!')

### Create the vector table schema

In [ ]:
# we are going to truncate the table
if db.has_table("raceday.vectors")["table_exists"]:
    db.clear_table(table_name="raceday.vectors")

schema = [
    ["ts_bkt"             , "string", "datetime"],
    ["TireSlipRatioFl_Vec", "bytes" , "vector(22)"],
    ["TireSlipRatioFr_Vec", "bytes" , "vector(22)"],
    ["TireSlipRatioRl_Vec", "bytes" , "vector(22)"],
    ["TireSlipRatioRr_Vec", "bytes" , "vector(22)"],
    ["AccelerationX_Vec"  , "bytes" , "vector(22)"],
    ["AccelerationY_Vec"  , "bytes" , "vector(22)"],
    ["AccelerationZ_Vec"  , "bytes" , "vector(22)"],
    ["VelocityX_Vec"      , "bytes" , "vector(22)"],
    ["VelocityY_Vec"      , "bytes" , "vector(22)"],
    ["VelocityZ_Vec"      , "bytes" , "vector(22)"],
    ["CombinedRace_Vec"   , "bytes" , "vector(220)"]
]

tableObj = gpudb.GPUdbTable(
    _type=schema,
    name="raceday.vectors",
    use_multihead_io=True,
    multihead_ingest_batch_size=5000,
    db=db
)

### Now that we have created our vector table, we need to loop through all the time-series data, convert to our 22-dimensional vector embedding space, then insert into Kinetica

In [ ]:
has_more = True
offset = 0
while has_more:
    result = None
    try:
        result = db.execute_sql('''select
       time_bucket(interval 5 second, timestamp) as ts_bkt,
       TireSlipRatioFl,
       TireSlipRatioFr,
       TireSlipRatioRl,
       TireSlipRatioRr,
       AccelerationX,
       AccelerationY,
       AccelerationZ,
       VelocityX,
       VelocityY,
       VelocityZ
    from
        raceday.dash1''',
                                encoding='json',
                                offset=offset)

    except gpudb.GPUdbException as gpudberror:
        print(str(gpudberror))
        
    if result is not None:
        j = json.loads(result.json_encoded_response)
        flat_result = {}
        count = 0

        has_more = result.has_more_records
        offset += len(j['column_1'])
        
        for count in range(0, len(j['column_1'])):
            if j['column_1'][count] not in flat_result:
                flat_result[j['column_1'][count]] = {
                    'TireSlipRatioFl_Vec': [],
                    'TireSlipRatioFr_Vec': [],
                    'TireSlipRatioRl_Vec': [],
                    'TireSlipRatioRr_Vec': [],
                    'AccelerationX_Vec':   [],
                    'AccelerationY_Vec':   [],
                    'AccelerationZ_Vec':   [],
                    'VelocityX_Vec':       [],
                    'VelocityY_Vec':       [],
                    'VelocityZ_Vec':       [],
                    'CombinedRace_Vec':    []
                }

            flat_result[j['column_1'][count]]['TireSlipRatioFl_Vec'].append(j['column_2'][count])
            flat_result[j['column_1'][count]]['TireSlipRatioFr_Vec'].append(j['column_3'][count])
            flat_result[j['column_1'][count]]['TireSlipRatioRl_Vec'].append(j['column_4'][count])
            flat_result[j['column_1'][count]]['TireSlipRatioRr_Vec'].append(j['column_5'][count])
            flat_result[j['column_1'][count]]['AccelerationX_Vec'].append(j['column_6'][count])
            flat_result[j['column_1'][count]]['AccelerationY_Vec'].append(j['column_7'][count])
            flat_result[j['column_1'][count]]['AccelerationZ_Vec'].append(j['column_8'][count])
            flat_result[j['column_1'][count]]['VelocityX_Vec'].append(j['column_9'][count])
            flat_result[j['column_1'][count]]['VelocityY_Vec'].append(j['column_10'][count])
            flat_result[j['column_1'][count]]['VelocityZ_Vec'].append(j['column_11'][count])
            
        for key, value in flat_result.items():
            for k2, v2 in value.items():
                if k2 != 'CombinedRace_Vec':
                    vec = c22(v2)
                    vec = [0 if math.isnan(x) else x for x in vec]
                    flat_result[key][k2] = vec
                    flat_result[key]['CombinedRace_Vec'].extend(vec)
                    
        for key, value in flat_result.items():
            json_record = json.dumps({'ts_bkt':              key,
                                      'TireSlipRatioFl_Vec': value['TireSlipRatioFl_Vec'],
                                      'TireSlipRatioFr_Vec': value['TireSlipRatioFr_Vec'],
                                      'TireSlipRatioRl_Vec': value['TireSlipRatioRl_Vec'],
                                      'TireSlipRatioRr_Vec': value['TireSlipRatioRr_Vec'],
                                      'AccelerationX_Vec':   value['AccelerationX_Vec'],
                                      'AccelerationY_Vec':   value['AccelerationY_Vec'],
                                      'AccelerationZ_Vec':   value['AccelerationZ_Vec'],
                                      'VelocityX_Vec':       value['VelocityX_Vec'],
                                      'VelocityY_Vec':       value['VelocityY_Vec'],
                                      'VelocityZ_Vec':       value['VelocityZ_Vec'],
                                      'CombinedRace_Vec':    value['CombinedRace_Vec']
                                      })
            response = db.insert_records_from_json(
                table_name='raceday.vectors',
                json_records=json_record)
tableObj.flush_data_to_server()

### Using SQLAssist to use vector similarity search

In [75]:
# Here you must ask a question relevant to the LLM context provided in the prompt template.
response: SqlResponse = chain.invoke({"input": 'find videos similar to https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-15.mp4'})
display(HTML(response.dataframe.to_html(index=False)))

ts_bkt,videopath,similarity
2023-10-30 19:27:50,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-2-15.mp4,1.192093e-07
2023-10-30 19:42:20,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-3-49.mp4,4.190129e-02
2023-10-30 19:24:00,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-1-03.mp4,6.220514e-02
2023-10-30 19:42:00,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-3-45.mp4,7.793921e-02
2023-10-30 19:40:55,https://kinetica-raceday.s3.amazonaws.com/raceday1/forza_images/output-3-32.mp4,8.653277e-02


# Interacting with Nemo
Now we want to do something interesting, so instead of just converting natural language to SQL, lets get two LLM's talking to each other

In [89]:
import importlib
import kinetica.kineai
from kinetica.kineai import SqlAssistLLM

importlib.reload(kinetica.kineai)
kineticallm = kinetica.kineai.KineticaLLM('raceday.raceday_ui_ctxt')

INFO:SqlAssistLLM:Connected to http://172.31.33.30:9191. (version 7.2.0.0)


### Create the context object

In [90]:
system = """ KineticAI is a cheerful AI assistant for engaging in a conversation between an LLM using the Nemo framework and the Kinetica LLM.  The Kinetica
LLM is designed to translate natural language questions into SQL queries. 

In addition to responding with  natural language it is able to ask questions to a database AI named SqlAssist that can query and summarize the logs. 
If it responds with a "KineticaLLM |  question" where question is sent to the SqlAssist AI. The SqlAssist AI will respond with an answer 
to the question in JSON format to the question made to SqlAssist by KineticAI.

when presented with a question, you should prefix your response with "KineticaLLM |  "
if a sentence ends in a "?", you should prefix your response with "KineticaLLM |  "

Consider the following example where a user asks KineticAI a question and KineticAI asks a followup question to SqlAssist. KineticAI uses the response from 
SqlAssist to answer the user's question.

user: what is the weather like today?
assistant: KineticaLLM |  what is the weather like today?
user: KineticaLLM |  [{"EXPR_0": 5.4}]
assistant: The answer is 5.4
"""

context0 = [dict(role="system", content=system),
            dict(role="user", content="what is the weather like today?"),
            dict(role="assistant", content="KineticaLLM |  what is the weather like today?"),
            dict(role="user", content="how many rows of data are you storing?"),
            dict(role="assistant", content="KineticaLLM |  how many rows of data are you storing?"),
            dict(role="user", content="what is the average number of telemetry rows per 5 second increment?"),
            dict(role="assistant", content="KineticaLLM |  what is the average number of telemetry rows per 5 second increment?"),
            dict(role="user", content="find me videos"),
            dict(role="assistant", content="KineticaLLM |  find me videos")]

# samples


In [92]:
question = 'what is the average velocity along the X axis?'
response = kineticallm.chat(context0, question)

INFO:NemoChatLLM:user: what is the average velocity along the X axis? (tokens: 780/3316)
INFO:NemoChatLLM:assistant: KineticaLLM |  what is the average velocity along the X axis? (tokens: 870/3226)
INFO:SqlAssistLLM:Query: what is the average velocity along the X axis?
INFO:SqlAssistLLM:SQL: SELECT
    avg(VelocityX)
FROM
    raceday.video_join1
INFO:NemoChatLLM:user: KineticaLLM |  [{"EXPR_0": 1.077429120258083}] (tokens: 803/3293)
INFO:NemoChatLLM:assistant: The answer is 1.077429120258083 (tokens: 897/3199)


### Progression
- without kinetica, nemo has no way to answer data question
- 